In [1]:
import os
import re
import nltk
import pandas as pd     
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import PlaintextCorpusReader
from textblob import TextBlob
from textblob.classifiers import NaiveBayesClassifier

In [2]:
# Carrega corpus

loc = '../discursos'
wordLists = PlaintextCorpusReader(loc, '.*\.txt')
print(wordLists.fileids())

['2020-03-03.txt', '2020-03-04.txt', '2020-03-09.txt', '2020-03-10.txt', '2020-04-17.txt', '2020-04-24.txt', '2020-04-29.txt', '2020-05-21.txt', '2020-06-05.txt', '2020-06-08.txt', '2020-06-09.txt', '2020-06-17.txt', '2020-06-22.txt', '2020-06-25.txt', '2020-06-30.txt', '2020-07-02.txt', '2020-07-16.txt', '2020-07-30.txt', '2020-07-31.txt', '2020-08-05.txt', '2020-08-06.txt', '2020-08-11.txt', '2020-08-12.txt', '2020-08-13.txt', '2020-08-14.txt', '2020-08-15.txt', '2020-08-17.txt', '2020-08-18.txt', '2020-08-19.txt', '2020-08-21.txt', '2020-08-24.txt', '2020-08-25.txt', '2020-08-26.txt', '2020-08-27.txt', '2020-08-29.txt', '2020-09-01.txt', '2020-09-02.txt', '2020-09-03.txt', '2020-09-10.txt', '2020-09-11.txt', '2020-09-16.txt', '2020-09-17.txt', '2020-09-18.txt', '2020-09-19.txt', '2020-09-22.txt']


In [3]:
# Carrega stopwords em português

stop_words = stopwords.words('portuguese')

In [4]:
# Carrega discursos para dataframe

fileid_list, text_list, words = [], [], []

for fileid in wordLists.fileids():
    words = [w for w in list(wordLists.words(fileid)) if w.isalpha() and not w.lower() in stop_words]
    text = " ".join(str(elem) for elem in words)
    fileid_list.append(fileid)
    text_list.append(text)
    
df = pd.DataFrame(fileid_list, columns=['filename'])
df['texts'] = text_list

df.head()

,filename,texts
0,2020-03-03.txt,Discurso Presidente República Jair Bolsonaro d...
1,2020-03-04.txt,Discurso Presidente República Jair Bolsonaro d...
2,2020-03-09.txt,Discurso Senhor Presidente República Jair Bols...
3,2020-03-10.txt,Discurso Presidente República Jair Bolsonaro d...
4,2020-04-17.txt,Discurso Presidente República Jair Bolsonaro d...


In [5]:
for text in df['texts']:
    print("{}\n\n\n".format(text))

Discurso Presidente República Jair Bolsonaro durante Solenidade Lançamento Programa Abrace Marajó Palácio PlanaltoPalácio Planalto março Senhores Senhoras Prezada Damares nome cumprimento todos prazer alegria grande estar frente governo onde todos integrantes têm compromisso deixar ninguém trás pedaço terra chamado Marajó pouco ouvimos falar sobre outras tantas tantas regiões Brasil pedaço terra tamanho estado Rio Janeiro população semelhante estado Roraima momento prezado almirante Bento ministro Minas Energia sofrendo falta construção linhão realidade ainda porque passado responsabilidade demarcaram vontade sabor interesses internacionais grandes reservas indígenas quantidade enorme quilombolas lutando contra abandonar índio quilombola integrá lo sociedade fazer tal medida possamos levar progresso todas regiões Brasil quero fazer confissão aqui prezado Éder Mauro colega Pará deu força mim lá Éder único estado nove estados região Nordeste onde ganhei demérito faz parte jogo democrátic

In [ ]:
# Treina modelo

corpus = pd.read_csv('../corpora/SentiLex-PT02/SentiLex-flex-PT02.csv', sep=';', names=["word", "pos", "flex", "tg", "sentiment", "anot"])
train, words, sentiments = [], [], []

for index, row in corpus.iterrows():
    sentiment_row = int(row['sentiment'].split("=")[1])
    if sentiment_row > 0:
        sentiment = 'pos'
    elif sentiment_row < 0:
        sentiment = 'neg'
    else:
        sentiment = 'neutral'
    word_row = row['word'].split(",")
    words.append(word_row[0])
    words.append(word_row[1])
    sentiments.append(sentiment)
    sentiments.append(sentiment)
    train.append((word_row[0], sentiment))
    train.append((word_row[1], sentiment))

cl = NaiveBayesClassifier(train)

In [ ]:
# Teste para a acurácia
# Trechos criados com palavras presentes no corpus

test = [
    ('Minha solidariedade!', 'pos'),
    ('o Brasil está abalado', 'neg'),
    ('Me sinto humilhado', 'neg'),
    ('É uma grande satisfação servi-los', 'pos'),
    ('O exército oferecerá socorro', 'pos'),
    ('É um sofrimento esta situação', 'neg')
]

In [ ]:
# Acurácia do modelo

cl.accuracy(test)

In [ ]:
# Características mais informativas do discurso

cl.show_informative_features(15)

In [ ]:
sentiments, probability_distribution, prob_dist_pos, prob_dist_neg, prob_dist_neutral = [], [], [], [], []

for text in df['texts']:
    sentiments.append(cl.classify(text))
    prob_dist = cl.prob_classify(text)
    probability_distribution.append(prob_dist.max())
    prob_dist_pos.append(round(prob_dist.prob("pos"), 2))
    prob_dist_neg.append(round(prob_dist.prob("neg"), 2))
    prob_dist_neutral.append(round(prob_dist.prob("neutral"), 2))
    
df['sentiment'] = sentiments
df['probability_distribution'] = probability_distribution
df['prob_dist_pos'] = prob_dist_pos
df['prob_dist_neg'] = prob_dist_neg
df['prob_dist_neutral'] = prob_dist_neutral

df.head()

In [ ]:
pos = [text for index, text in enumerate(df['texts']) if df['sentiment'][index] == 'pos']
neg = [text for index, text in enumerate(df['texts']) if df['sentiment'][index] == 'neg']
neutral = [text for index, text in enumerate(df['texts']) if df['sentiment'][index] == 'neutral']

In [ ]:
chart = pd.Series((len(pos), len(neg), len(neutral)), index=('Positivos', 'Negativos', 'Neutros'), name='Análise de Sentimentos')
chart.plot.pie(fontsize=12, autopct='%.2f', figsize=(5, 5));